# NMSU CSCI-5435 Assignment 1

## Relevent Information

In [1]:
#Name:               Tianjie Chen
#Email:              tvc5586@nmsu.edu
#File Creation Date: Jan/30/2025
#Purpose of File:    NMSU CSCI-5435 Assignment 1
#Last Edit Date:     Jan/30/2025
#Last Edit Note:     Adding comments
#GenAI used:         False

## Setting up environment

### Hugging Face Login

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Assignment Task 1 & 2

### Load Dataset

In [3]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("yelp_review_full")

Using the latest cached version of the dataset since yelp_review_full couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'yelp_review_full' at /home/tchen/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/0.0.0/c1f9ee939b7d05667af864ee1cb066393154bf85 (last modified on Thu Jan 30 07:59:43 2025).


### Set up training corpus

In [4]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["text"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

### Load tokenizer

In [5]:
from transformers import BertTokenizerFast

old_tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

### Test untrained tokenizer

In [6]:
example = "I miss this place. The grilled cheese was top notch. The fries were good and the vanilla shake was yummy."

In [7]:
tokens = old_tokenizer.tokenize(example)
print(tokens)
print(len(tokens))

['I', 'miss', 'this', 'place', '.', 'The', 'g', '##rille', '##d', 'cheese', 'was', 'top', 'notch', '.', 'The', 'f', '##ries', 'were', 'good', 'and', 'the', 'van', '##illa', 'shake', 'was', 'y', '##um', '##my', '.']
29


### Train tokenizer on loaded dataset

In [8]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

### Test trained tokenizer

In [9]:
tokens = tokenizer.tokenize(example)
print(tokens)
print(len(tokens))

['I', 'miss', 'this', 'place', '.', 'The', 'grilled', 'cheese', 'was', 'top', 'notch', '.', 'The', 'fries', 'were', 'good', 'and', 'the', 'vanilla', 'shake', 'was', 'yummy', '.']
23


## Assignment Task 3

### Create the main function

In [10]:
import sys
from Levenshtein import distance

"""
A function that generates the most likely words
based on the provided input token and vocabulary

Parameters:
    in_string : string
        The input token
    vocab : dict
        The vocabulary

Return:
    suggestions : list (2D)
        A list that contains the top 5 most likely
        words from the vocabulary and their
        corresponding edit distances. Each word and 
        its edit distance is grouped into a list
"""
def get_spelling_suggestions(in_string, vocab):
    suggestions = [
        ["", sys.maxsize],
        ["", sys.maxsize],
        ["", sys.maxsize],
        ["", sys.maxsize],
        ["", sys.maxsize],
    ]

    # for each token in the vocabulary
    for token, _ in vocab.items():
        # for each token-edit distance pair in the suggestions
        for i in suggestions:
            # Using levenshtein distance
            edit_distance = distance(in_string, token, weights=(1, 1, 2))

            # if token is already in the suggestions
            if token == i[0]:
                break

            # if token's edit distance is less than what's in 
            # the suggestions
            elif edit_distance < i[1]:
                i[0] = token
                i[1] = edit_distance
                break

    return suggestions

### Create helper function for printing

In [11]:
"""
A helper function for printing the results from 
the main function

Parameters:
    in_string : string
        The same input token provided to the 
        main function
    suggestions : list (2D)
        The suggestions provided by the main 
        function
"""
def print_suggestions(in_string, suggestions):
    print(f"The top 5 suggestions for correcting {in_string} are:")
    
    for suggestion, edit_distance in suggestions:
        print(f"{suggestion} with edit distance of {edit_distance}")

    print("")

### Test the main function

In [12]:
test_words = ["luckilq", "proffes", "skyises", "dismayo"]

# test each word in the test_words list
for word in test_words:
    print_suggestions(word, get_spelling_suggestions(word, tokenizer.vocab))

The top 5 suggestions for correcting luckilq are:
luckily with edit distance of 2
luck with edit distance of 3
kil with edit distance of 4
luc with edit distance of 4
lucky with edit distance of 4

The top 5 suggestions for correcting proffes are:
profess with edit distance of 2
profiles with edit distance of 3
pres with edit distance of 3
offs with edit distance of 3
pros with edit distance of 3

The top 5 suggestions for correcting skyises are:
skies with edit distance of 2
skis with edit distance of 3
kiss with edit distance of 3
skys with edit distance of 3
sites with edit distance of 4

The top 5 suggestions for correcting dismayo are:
dismay with edit distance of 1
mayo with edit distance of 3
dism with edit distance of 3
dismayed with edit distance of 3
may with edit distance of 4

